# Disciplina: Introdução à Teoria da Informação 

## Projeto (Parte 1): Compressor e Descompressor de dados com LZW

### Dupla: Gabriel Patrício (20170170889) e Sarah Carvalho (20170022975)


**Descrição do Projeto**: 

Implemente um compressor e descompressor com o algoritmo LZW. Considere que as
mensagens são geradas por fontes com alfabeto A = {0, 1, ..., 255}. Teste o
compressor/descompressor com um corpus de texto em português de 16MB e com um
arquivo binário de vídeo.

O índice do dicionário deve ser testado com diferentes tamanhos K bits
(parâmetro). Exemplo: K=9bits tamanho do dicionário: 2^9=512, K=
10bits tamanho do
dicionário 2^10
=1024. No relatório apresente as curvas de
RC x K e de Tempo de
Processamento x K, para K = 9, 10,
11, 12, 13, 14, 15,
16 bits. Indique também a
quantidade total de índices presentes na mensagem final para cada K.

**_Observações:_**

•
Os símbolos do arquivo de teste devem ser lidos no modo binário (número
s) e não
no modo texto (caracteres/strings).

•
O codificador deve receber como entrada um arquivo e gerar c
omo saída o
arquivo codificado

In [1]:
import binascii
import time
import os
import sys 
import struct

In [2]:
corpus= './database/corpus16MB.txt';
teste = './database/teste.txt';
video = './database/video.mp4';
file_selected = corpus

In [3]:
bin_data = open(file_selected, "rb").read()
data_size=len(bin_data)

In [4]:
bin_data = bin_data.decode('latin1')

## Criando o dicionário

In [5]:
K = 9 #Change this line to modify the dictionary size  
dictionary_size_half = int((2 ** K)/2)
dictionary_size_full = 2 ** K   


In [6]:
dictionary = {i.to_bytes(1, byteorder='big'): i for i in range(0,256)} #represents unicode code point of the character) 

In [7]:
dict_keys = list(dictionary.keys())
s=0
aa = []
for i in range(0, 256):
    aa.append(''.join(dict_keys[i].decode('latin1')))


In [8]:
codificado_char=[]
codificado_int=[]

## Codificando

In [9]:
j=dictionary_size_half
i=0
a=[]
count=0
y=bin_data[0]
rs = []
k=0
inicio = time.time()

while((i < data_size)):
    
    aux = bin_data[i]
    if (aux in aa):
        count=i 
        k=aa.index(aux) #Salva o indice do caracter
        x = aux

        while (x in aa): #Lógica para concatenação dos caracteres e checagem no dicionario
            count+=1
            if(count<data_size):
                y=x;
                k=aa.index(aux) #Salva o indice do caracter
                aux = bin_data[count]
                a = aux
                x+=a
            else:
                break
        if(x not in aa):
            if(j<dictionary_size_full):
                aa.append(x)
            j+=1 #Adicionou um indice
        else:
            y=x
        k=aa.index(y)
        codificado_int.append(k)
        codificado_char.append(y)
                
        i+=len(y) #Avança um caracter
        
    else: 
        if(j<dictionary_size_full):
            aa.append(aux)
        j+=1 #Adicionou um indice
        i+=1 #Avança um caracter

fim = time.time()

In [10]:
print("Demorou:", ((fim-inicio)/60), "minutos")

Demorou: 6.783687222003937 minutos


In [11]:
compressed_file = open(f'./compressed_files/{file_selected[11:-4]}_K_{K}.bin', 'wb')
compressed_file.write(struct.pack(
            f">{'H'*len(codificado_int)}", *codificado_int))
compressed_file.close()

In [12]:
indices_codificados = len(codificado_int)   
print("Tamanho do arquivo antes da compressão ", os.path.getsize(file_selected), "B")

Tamanho do arquivo antes da compressão  15637070 B


In [13]:
print("Quantidade de índices codificados: ", indices_codificados)
print("Tamanho esperado:", indices_codificados*2, 'B')
print("Tamanho obtido  :", os.path.getsize(f'./compressed_files/{file_selected[11:-4]}_K_{K}.bin'), 'B')

Quantidade de índices codificados:  9157927
Tamanho esperado: 18315854 B
Tamanho obtido  : 18315854 B
